# Trabalho Final - Modelos de IA e Machine Learning

### Integrantes:

###    1) Jurandir Ventura - RM336634
###    2) Marcelo Preto - RM336632
###    3) Fabio Rizzi - RM336656
###    4) Thiago Alexandre – RM336583

# Exercício 2.1

Utilizando o arquivo UsuariosTempoMedioQuantidade.csv, que é uma amostra do tempo de utilização dos usuários em um determinado sistema, faça a clusterização utilizando k means e DBSCAN.
a) O que representam os clusters gerados?

b) Qual é a melhor quantidade de Clusters?

c) Qual é a melhor técnica (k means ou DBSCAN) a ser utilizada nessa clusterização

In [4]:
import pandas as pd

In [5]:
#Leitura do dataset
dataset = pd.read_csv('data/2.1_usuarios-tempomedioquantidade.csv', sep=';', engine='python')
dataset.head()

,NomeUsuario,Login,Email,NomeEquipe,Quantidade,TempoMedio
0,dadostab1,dadostab1,dadostab1@dadospesquisa.com.br,Equipe Dados Pesquisa,1498,0
1,axistab2,axistab2,axistab2@axiscriativa.com.br,Equipe Axis Criativa,461,75
2,axistab1,axistab1,axistab1@axiscriativa.com.br,Equipe Axis Criativa,445,7
3,planocdetab1,planocdetab1,planocdetab1@planocde.com.br,Equipe PlanoCDE,423,140
4,Usuario11,usuario11,usuario11@marketnnova.com,Equipe Marketnnova,351,22
